# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?


The target is discrete. Binary specificially. The only outcomes are pass or fail. Therefore this is a classification problem.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [195]:
# Import libraries
import numpy as np
import pandas as pd

In [196]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns
seed = 1234

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [197]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = [x == "yes" for x in student_data["passed"]].count(True)
n_failed = [x == "yes" for x in student_data["passed"]].count(False)
grad_rate = float(n_passed) / (n_passed + n_failed)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 0.67%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [198]:
# Extract feature (X) and target (y) columns
print student_data.columns
print type(student_data.columns)
print student_data.index
print type(student_data.index)
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Index([u'school', u'sex', u'age', u'address', u'famsize', u'Pstatus', u'Medu',
       u'Fedu', u'Mjob', u'Fjob', u'reason', u'guardian', u'traveltime',
       u'studytime', u'failures', u'schoolsup', u'famsup', u'paid',
       u'activities', u'nursery', u'higher', u'internet', u'romantic',
       u'famrel', u'freetime', u'goout', u'Dalc', u'Walc', u'health',
       u'absences', u'passed'],
      dtype='object')
<class 'pandas.indexes.base.Index'>
RangeIndex(start=0, stop=395, step=1)
<class 'pandas.indexes.range.RangeIndex'>
Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18      

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [199]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [200]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn import cross_validation
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_all,y_all,train_size=300,random_state=seed)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [204]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import tree 
clf_dt = tree.DecisionTreeClassifier(random_state=seed)

# Fit model to training data
train_classifier(clf_dt, X_train, y_train)  # note: using entire training set here
print clf_dt.get_params()  # you can inspect the learned model by printing it

# No easy way to find out depth of tree after
# Need to traverse tree to find depth
# Instead, print to dot file here
from sklearn.externals.six import StringIO
from IPython.display import Image
with open("decisiontree.dot",'w') as f:
    f = tree.export_graphviz(clf_dt, out_file=f)

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
{'presort': False, 'splitter': 'best', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'criterion': 'gini', 'random_state': 1234, 'max_features': None, 'max_depth': None, 'class_weight': None}


In [218]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score,accuracy_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    print "Accuracy score: {}".format(accuracy_score(target.values, y_pred))
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf_dt, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Accuracy score: 1.0
F1 score for training set: 1.0


In [219]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf_dt, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
Accuracy score: 0.657627118644
F1 score for test set: 0.74686716792


In [220]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_all,y_all,train_size=300,random_state=seed)
train_predict(clf_dt, X_train, y_train, X_test, y_test)
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_all,y_all,train_size=200,random_state=seed)
train_predict(clf_dt, X_train, y_train, X_test, y_test)
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_all,y_all,train_size=100,random_state=seed)
train_predict(clf_dt, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Accuracy score: 1.0
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Accuracy score: 0.673684210526
F1 score for test set: 0.752
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Accuracy score: 1.0
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Accuracy score: 0.610256410256
F1 score for test set: 0.688524590164
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels us

In [221]:
# TODO: Train and predict using two other models
from sklearn import svm
clf_svc = svm.SVC()
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_all,y_all,train_size=300,random_state=seed)
train_predict(clf_svc, X_train, y_train, X_test, y_test)
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_all,y_all,train_size=200,random_state=seed)
train_predict(clf_svc, X_train, y_train, X_test, y_test)
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_all,y_all,train_size=100,random_state=seed)
train_predict(clf_svc, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.006
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
Accuracy score: 0.803333333333
F1 score for training set: 0.871459694989
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
Accuracy score: 0.684210526316
F1 score for test set: 0.797297297297
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.002
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
Accuracy score: 0.8
F1 score for training set: 0.87012987013
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
Accuracy score: 0.676923076923
F1 score for test set: 0.797427652733
------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
Accuracy score: 0.81
F1 score for training se

In [222]:
from sklearn.naive_bayes import GaussianNB
clf_nb = GaussianNB()
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_all,y_all,train_size=300,random_state=seed)
train_predict(clf_nb, X_train, y_train, X_test, y_test)
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_all,y_all,train_size=200,random_state=seed)
train_predict(clf_nb, X_train, y_train, X_test, y_test)
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_all,y_all,train_size=100,random_state=seed)
train_predict(clf_nb, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
Accuracy score: 0.72
F1 score for training set: 0.79512195122
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
Accuracy score: 0.705263157895
F1 score for test set: 0.794117647059
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
Accuracy score: 0.72
F1 score for training set: 0.798561151079
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
Accuracy score: 0.692307692308
F1 score for test set: 0.779411764706
------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.00

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [265]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn import grid_search
parameters = {'kernel':('linear','rbf'), 'C':[1,2,3,5,10]}
clf_svc_grd = grid_search.GridSearchCV(svm.SVC(), parameters, cv=3)
clf_svc_grd.fit(X_train,y_train)

print clf_svc_grd.best_estimator_
print "%s (mean)" % clf_svc_grd.best_score_
print clf_svc_grd.best_params_
print clf_svc_grd.grid_scores_
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf_svc_grd, X_test, y_test))
print

parameters = {'max_depth':(1,2,3,4,5,6,7,8,9,10),'criterion':('gini','entropy')}
#scoring_function = make_scorer(mean_squared_error,False)
clf_dt_grd = grid_search.GridSearchCV(tree.DecisionTreeClassifier(random_state=seed),parameters, cv=6)
clf_dt_grd.fit(X_train, y_train)

best_estimator = clf_dt_grd.best_estimator_
with open("decisiontree_grd.dot",'w') as g:
    g = tree.export_graphviz(best_estimator, out_file=g)

print best_estimator
print "%s (mean)" % clf_dt_grd.best_score_
print clf_dt_grd.best_params_
print clf_dt_grd.grid_scores_
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf_dt_grd, X_test, y_test))

SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.7 (mean)
{'kernel': 'rbf', 'C': 2}
[mean: 0.60000, std: 0.03920, params: {'kernel': 'linear', 'C': 1}, mean: 0.69000, std: 0.01678, params: {'kernel': 'rbf', 'C': 1}, mean: 0.54000, std: 0.00756, params: {'kernel': 'linear', 'C': 2}, mean: 0.70000, std: 0.02510, params: {'kernel': 'rbf', 'C': 2}, mean: 0.52000, std: 0.04994, params: {'kernel': 'linear', 'C': 3}, mean: 0.66000, std: 0.01707, params: {'kernel': 'rbf', 'C': 3}, mean: 0.52000, std: 0.05466, params: {'kernel': 'linear', 'C': 5}, mean: 0.64000, std: 0.00504, params: {'kernel': 'rbf', 'C': 5}, mean: 0.57000, std: 0.03933, params: {'kernel': 'linear', 'C': 10}, mean: 0.62000, std: 0.01248, params: {'kernel': 'rbf', 'C': 10}]
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.002
Accurac